In [0]:
%sh

( cd /Volumes/edsm/bronze/utility && rm -rvf * )
( cd /Volumes/edsm/silver/utility && rm -rvf * )
( cd /Volumes/edsm/gold/utility   && rm -rvf * )

In [0]:
catalog = dbutils.widgets.get("catalog")
schemas = spark.sql(f"SHOW SCHEMAS IN {catalog}").collect()
for record in schemas:
    schema = record.databaseName
    if schema in [ "default", "information_schema", "pg_catalog" ]:
        continue
    tables = spark.sql(f"SHOW TABLES IN {catalog}.{schema}").collect()
    for tbl in tables:
        if tbl.isTemporary:
            continue
        table_name = tbl.tableName
        desc = spark.sql(f"DESCRIBE EXTENDED {catalog}.{schema}.{table_name}").collect()
        is_view = any("Type" in row.col_name and "VIEW" in row.data_type for row in desc)
        if is_view:
            continue
        print(f"Dropping {catalog}.{schema}.{tbl.tableName}")
        spark.sql(f"DROP TABLE {catalog}.{schema}.{tbl.tableName}")